In [2]:
import sys
import os
import numpy as np

sys.path.append(os.path.abspath(os.path.join('..')))

from pipeline.parameters import KlausmeierParameters

In [6]:
params = KlausmeierParameters(
    A=2.0,    # Opady
    L=1.0,    # Parowanie
    R=0.5,    # Pobór wody
    J=1.0,    # Konwersja
    M=0.45,   # Śmiertelność
    Dw=0.1,   # Dyfuzja wody
    Dn=0.01   # Dyfuzja roślin
)

wyniki = params.get_dimensionless_params()

print("parametry bezwymiarowe:")
print(f"a (opady) = {wyniki['a']:.4f}")
print(f"m (śmiertelność) = {wyniki['m']:.4f}")
print(f"d1 = {wyniki['d1']:.4f}")
print(f"d2 = {wyniki['d2']:.4f}")

parametry bezwymiarowe:
a (opady) = 1.4142
m (śmiertelność) = 0.4500
d1 = 0.1000
d2 = 0.0100


# 1. Analiza matematyczna - ubezwymiarowienie

Chcemy zredukować liczbę parametrów modelu poprzez wprowadzenie zmiennych bezwymiarowych. Wyjściowy model Klausmeiera opisuje zmiany wody ($W$) i biomasy ($N$) w czasie ($T$) i przestrzeni ($X$):

$$
\begin{cases}
\displaystyle \frac{\partial W}{\partial T} = A - LW - RWN^2 + D_W \Delta W \\[1em]
\displaystyle \frac{\partial N}{\partial T} = JRWN^2 - MN + D_N \Delta N
\end{cases}
\tag{1}
$$

Wprowadzamy następujące zmienne bezwymiarowe, skalując wielkości fizyczne przez stałe $W_0, N_0, T_0$:

$$
W = W_0 u, \quad N = N_0 v, \quad T = T_0 t \tag{2}
$$

Zakładamy $X_0 = 1$, zachowując oryginalne jednostki długości.

Wychodzimy od fizycznego równania ewolucji biomasy ($N$):

$$
\frac{\partial N}{\partial T} = J R W N^2 - M N + D_N \Delta N
$$

Stosujemy podstawienia z równań (2):

$$
\frac{N_0}{T_0} \frac{\partial v}{\partial t} = J R (W_0 u) (N_0 v)^2 - M (N_0 v) + D_N \Delta (N_0 v)
$$

Po uporządkowaniu wyrazów dostajemy:

$$
\frac{\partial v}{\partial t} = (J R W_0 N_0 T_0) u v^2 - (M T_0) v + D_N T_0 \Delta v
$$

Podstawiając zależności do równania na wodę, otrzymujemy analogicznie:

$$
\frac{W_0}{T_0} \frac{\partial u}{\partial t} = A - L W_0 u - R W_0 N_0^2 u v^2 + W_0 D_W \Delta u
$$

Dzielimy obustronnie przez czynnik stojący przy pochodnej ($W_0/T_0$), co daje:

$$
\frac{\partial u}{\partial t} = \frac{A T_0}{W_0} - (L T_0) u - (R T_0 N_0^2) u v^2 + D_W T_0 \Delta u
$$

Przy wyrazach $u$ oraz $uv^2$ pojawiają się grupy stałych. Sprowadzamy równanie do postaci kanonicznej:

1. $L T_0 = 1 \implies T_0 = 1/L$ (skala czasu)
2. $R T_0 N_0^2 = 1 \implies N_0 = \sqrt{L/R}$ (skala biomasy)
3. $J R W_0 T_0 N_0 = 1 \implies W_0 = \frac{\sqrt{LR}}{JR}$ (skala wody)

Po zastosowaniu tych podstawień definujemy nowe, bezwymiarowe stałe sterujące systemem:

$$
a = \frac{A}{L} \frac{JR}{\sqrt{LR}}, \quad m = \frac{M}{L}, \quad d_1 = \frac{D_W}{L}, \quad d_2 = \frac{D_N}{L}
\tag{3}
$$

Ostateczny układ równań, który będziemy badać:

$$
\begin{cases}
\displaystyle \frac{\partial u}{\partial t} = a - u - uv^2 + d_1 \Delta u & (4) \\[1em]
\displaystyle \frac{\partial v}{\partial t} = uv^2 - mv + d_2 \Delta v & (5)
\end{cases}
$$